Actions action = new Actions(driver);
WebElement nametxt = driver.findElement(By.id("name"));
action.moveToElement(nametxt).click().sendKeys("Test Name").perform();

In [1]:
from selenium.webdriver import Chrome, ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import visibility_of_all_elements_located
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import datetime
import os
import re
import time
import json
import subprocess
import glob
import calendar

today = datetime.date.today()

In [2]:
CLOUD_GATE = r'https://echizen.cloudgate.jp/user-hub/yachiyo-ind/'

GMAIL_URL = r'https://mail.google.com/a/yachiyo-ind.co.jp/'
CALENDAR_URL = r'https://calendar.google.com/calendar/u/0/r?tab=mc&pli=1'
KOBETSU_GENKA_URL = r'https://echizen.cloudgate.jp/sso/yachiyo-ind/post.xhtml?providerUuid=284c5466-7256-49e3-99ab-04ff1a8178a8'
KINTAIKANRI_URL = r'https://yachiyo-ind.obic7.obicnet.ne.jp/Obic7FederationService'
GUEST_WIFI_URL = r'https://sites.google.com/yachiyo-ind.co.jp/yachiyo-portal/guest-wifi'

In [3]:
def start_google_chrome_with_port(url):
    
    chromePath = r'"C:\Program Files\Google\Chrome\Application\chrome.exe" -remote-debugging-port=9222 --user-data-dir=C:\Temp_Chrome'
    
    subprocess.Popen(chromePath)
    time.sleep(3)

    options = ChromeOptions()
    options.use_chromium = True
    options.debugger_address = r'127.0.0.1:9222'
    
    try:
        chromedriverPath = ChromeDriverManager().install()
    except:
        chromedriverPath = 'C:\python\chromedriver\chromedriver.exe'
    
    service = Service(executable_path=chromedriverPath)
    driver = Chrome(service=service, options=options)
    actionChains = ActionChains(driver)
    
    # driver.set_window_position(1920, 0)
    driver.set_window_size(1300, 1000)
    driver.implicitly_wait(5)
    
    driver.get(url)
    
    return driver, actionChains



In [4]:
def send_keys_and_enter(driver, loginInfo, css, keys):
    
    wait = WebDriverWait(driver, 10)
    wait.until(visibility_of_all_elements_located((By.CSS_SELECTOR, css)))
    
    eles = driver.find_elements(By.CSS_SELECTOR, css)
    if eles:
        eles[0].clear()
        eles[0].send_keys(loginInfo[keys])
        eles[0].send_keys(Keys.ENTER)
        time.sleep(3)


def open_new_window(driver, url, css, winNum):
    
    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[winNum])
    driver.get(url)
    
    if url == GMAIL_URL:
        try:
            eles = driver.find_elements(By.CSS_SELECTOR, 'div > div > div > div > button > span')
            if eles:
                eles[0].click()
        except:
            pass
        
    wait = WebDriverWait(driver, 30)
    wait.until(visibility_of_all_elements_located((By.CSS_SELECTOR, css)))
        
    if url == KOBETSU_GENKA_URL:
        eles = driver.find_elements(By.CSS_SELECTOR, '#bodyMain > header > nav > a')
        if eles:
            eles[0].click()
        eles = driver.find_elements(By.CSS_SELECTOR, 'section > ul > li:nth-child(2) > a')
        if eles:
            eles[0].click()

In [5]:
try:
    with open(glob.glob('../*.json')[0]) as f:
        loginInfo = json.load(f)
except FileNotFoundError as e:
    print(e)

In [6]:
driver, actionChains = start_google_chrome_with_port(CLOUD_GATE)

In [7]:
try:
    send_keys_and_enter(driver, loginInfo, '#rawUsername', 'ID')
    send_keys_and_enter(driver, loginInfo, '#password', 'PASSWORD')
except:
    pass

In [8]:
newWindosList = {
    # GMAIL_URL: '#\:mr > div > div > a',
    # CALENDAR_URL: 'body',
    KOBETSU_GENKA_URL: 'body',
    # KINTAIKANRI_URL: 'body',
}

In [9]:
for winNum, (url, css) in enumerate(newWindosList.items()):
    open_new_window(driver, url, css, winNum+1) 

In [10]:
# 日別合計
def extact_daily_total(rowNum):
    
    get_table = []
    
    css = f'#tblDayTotal > tbody > tr:nth-child({rowNum}) > td'
    for ele in driver.find_elements(By.CSS_SELECTOR, css):
        txt = ele.text
        if txt == '':
            get_table.append(0.0)
        else:
            get_table.append(txt)
            
    css = '#tblDayTotal_wrapper > div:nth-child(3) > div > div > div > div > table > thead > tr'
    index = ['index'] + driver.find_elements(By.CSS_SELECTOR, css)[0].text.split()
    
    return get_table, index

上段テーブルの取得

In [11]:
get_tables = []
for rowNum in range(5):
    get_table, columns = extact_daily_total(rowNum+1)
    get_tables.append(get_table)

日付のインデックスを作成

In [12]:
firstDate = today.replace(day=1)
lastDate = today.replace(day=calendar.monthrange(today.year, today.month)[1])
date_index = pd.date_range(start=firstDate, end=lastDate, freq="d")

DetaFrameの整列

In [13]:
initial_df = pd.DataFrame(get_tables).T
initial_df.columns = list(initial_df.iloc[0])
df = initial_df.drop(index=initial_df.index[[0, 1]])
df = df.drop(columns=initial_df.columns[-1], axis=1)
df = df.astype(float)
df = df.set_index(date_index)
df.head(2)

,要員工数,設備工数,勤怠工数,勤怠-要員
2023-06-01,0.0,0.0,9.2,9.2
2023-06-02,0.0,0.0,9.2,9.2


下のテーブルを取得 代入はしていない

In [14]:
def extract_row_data(rowNum):
    rowData = []
    css = f'#divTblProductCostInfo > div > div > div > div > table > tbody > tr:nth-child({rowNum}) > td'
    for ele in (driver.find_elements(By.CSS_SELECTOR, css)):
        txt = ele.text
        if txt == '':
            rowData.append(None)
        else:
            rowData.append(txt)
    return rowData

# ズームコントロールがうまくいかない
#driver.execute_script("document.body.style.zoom='30%'")

table = []
for i in range(7):
    rowData = extract_row_data(i+1)
    table.append(rowData)
    
inputTable = pd.DataFrame(table)

# 開発管理No.
inputTable.iloc[:, 3]

0    KP01YRSX00002
1    KP02YRSX00002
2    YRD0000000000
3    YRD0000000000
4    YRD0000000000
5    YRD0000000000
6    YRD0000000000
Name: 3, dtype: object

工数の整理

In [15]:
# 不稼働表示 'YRD-0', 'YRD-1', 'YRD-2', 'YRD-3', 'YRD-4'
mainOperationtime = 4.0
downtime = 1.0
df['KP01YRSX02'] = mainOperationtime
df['KP02YRSX02'] = (df['勤怠工数'] - mainOperationtime - downtime).round()

for i, label in enumerate(['YRD-0', 'YRD-1', 'YRD-2', 'YRD-3', 'YRD-4']):
    df.loc[df.index.weekday == i, label] = downtime

df.loc[df.index.weekday == 5, ['KP01YRSX02', 'KP02YRSX02']] = 0
df.loc[df.index.weekday == 6, ['KP01YRSX02', 'KP02YRSX02']] = 0

df.fillna(0, inplace=True)
df.head(6)

,要員工数,設備工数,勤怠工数,勤怠-要員,KP01YRSX02,KP02YRSX02,YRD-0,YRD-1,YRD-2,YRD-3,YRD-4
2023-06-01,0.0,0.0,9.2,9.2,4.0,4.0,0.0,0.0,0.0,1.0,0.0
2023-06-02,0.0,0.0,9.2,9.2,4.0,4.0,0.0,0.0,0.0,0.0,1.0
2023-06-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-05,0.0,0.0,0.0,0.0,4.0,-5.0,1.0,0.0,0.0,0.0,0.0
2023-06-06,0.0,0.0,0.0,0.0,4.0,-5.0,0.0,1.0,0.0,0.0,0.0


昨日までを入力

In [16]:
df_yesterday = df[df.index < today.strftime('%Y%m%d')]

for x, column in enumerate(df_yesterday.index):
    for y, index in enumerate(df_yesterday.iloc[x, 4:]):
        css = f'#divTblProductCostInfo > div > div > div > div > table > tbody > tr:nth-child({1+y}) > td:nth-child({14+x})'
        ele = driver.find_element(By.CSS_SELECTOR, css)
        actionChains.click(ele).perform()
        actionChains.send_keys(Keys.BACKSPACE).perform()
        actionChains.send_keys(f'{index}').perform()

登録ボタンを押す

In [19]:
driver.find_elements(By.CSS_SELECTOR, '#btnReg')[0].click()

In [30]:
driver.find_elements(By.CSS_SELECTOR, 'button.swal2-confirm.swal2-styled')[0]

<selenium.webdriver.remote.webelement.WebElement (session="09cdf34197ee2ad9c21dd9ca5c8a6ee1", element="0777E2E62E7C8728B13689FAE3344456_element_2532")>

In [ ]:
# ynDict={'y':True,'yes':True,'n':False,'no':False}
# print('Would you like to register?')
# if ynDict[input('[Y]es/[N]o? >> ').lower()]:
#     driver.find_elements(By.CSS_SELECTOR, '#btnReg')[0].click()
# else:
    
#     pass

Would you like to register?
